In [296]:
# 稳定币网格策略到高频做市
# 加密货币的稳定币和美元挂钩，一般上下波动在千分之1到万分之1不等，所以其价格会在1美元上下波动，最终也会回到1
# 有些交易所的稳定币零手续费，例如binance，这是高频交易实战的很好的战场

# 网格策略 核心事在一个价格网格里面，低卖高卖，赚取差价，适用于震荡行情，也就是不断上下波动的行情，不适用于持续牛市和持续熊市
# 
# 具体逻辑：中线为价格=1，每条线差额为0.0002,以上下4条线的网格为例子，买入卖出仓位的百分比可以自己定
# 第一次买入点一般不得大于中线
#  -------------------------------------------------------------------------------------------------------上界线
# .....
#  -------------------------------------------------------------------------------------------------------price=1.0008
#  -------------------------------------------------------------------------------------------------------price=1.0004
#  -----------------------------4卖出------------------------------------10卖出----------------------------price=1.0002
#  -1买入---------3卖出---------------5买入-------------------9卖出----------------------------------------------------price=1.0000 中线
#  -------2买入---------------------------6买入---------8卖出----------------------------------------------price=0.9998
#  ----------------------------------------------7买入----------------------------------------------------price=0.9996
#  ------------------------------------------------------------------------------------------------------price=0.9994
# ......
#  ---------------------------------------------------------------------------------------下界线
#

In [297]:
import numpy as np
import pandas as pd
import vectorbt
import ccxt
import time
import plotly.graph_objects as go
import warnings

warnings.filterwarnings("ignore")

pd.set_option('display.max_columns', None)
pd.set_option('display.width', 2000)

In [298]:
class StrategyFrame():
    df = pd.DataFrame()
    exchange = ccxt.binance({
        'apiKey': '',
        'secret': '',
        'timeout': 30000,
        'options': {'defaultType': 'spot'},
        'proxies': {
            'http': 'http://127.0.0.1:7890',
            'https': 'http://127.0.0.1:7890',
        },
    })

    def __init__(self, data: pd.DataFrame):
        self.df = data

    # 获取最近的数据，要求在1000分钟之内
    def get_last_diff_data(self, symbol):
        since = self.df['t'].tail(1).values[0]
        print(f'since: {since}', pd.to_datetime(since, unit='ms', origin='1970-01-01 08:00:00'))
        res_df = pd.DataFrame(self.exchange.fetch_ohlcv(symbol, timeframe='1m', since=since))
        res_df.columns = ['t', 'o', 'h', 'l', 'c', 'v']
        res_df['datetime'] = pd.to_datetime(res_df['t'], unit='ms', origin='1970-01-01 08:00:00')
        res_df.set_index('datetime', inplace=True)
        self.df = pd.concat([self.df[:-1], res_df])
        return self

    def get_last_data(self, symbol, count=1):
        df = pd.DataFrame(self.exchange.fetch_ohlcv(symbol, timeframe='1m', limit=1000))
        for i in range(count - 1):
            since = df[0][0] - 1000 * 60 * 1000
            res_df = pd.DataFrame(self.exchange.fetch_ohlcv(symbol, timeframe='1m', since=since, limit=1000))
            df = pd.concat([res_df, df], ignore_index=True)
            time.sleep(0.2)
        df.columns = ['t', 'o', 'h', 'l', 'c', 'v']
        df['datetime'] = pd.to_datetime(df['t'], unit='ms', origin='1970-01-01 08:00:00')
        df.set_index('datetime', inplace=True)
        #
        if self.check_data(df):
            df.to_csv('./data.csv')
            self.df = df
        else:
            print('check data error,no update')
        return self

    #检查数据完整性 True完整 False不完整
    @staticmethod
    def check_data(data: pd.DataFrame):
        duplicated_ok = data[data.index.duplicated()].empty
        diff = data['t'].diff()[1:]
        diff_ok = len(diff[diff == diff.iloc[0]]) == len(diff)
        if not duplicated_ok:
            print('数据有重复')
        if not diff_ok:
            print('数据不连续')
        return duplicated_ok and diff_ok

    ###
    ## 使用网格策略
    # 假设grid有10个区间,具体操作如下
    # 1 第一次头寸，分析当前价格是否小于中线，小于直接买入50%
    # 2 价格基于每下降一格，继续买入10% 
    # 3 价格每上升一格，卖出10%
    # 4 若卖出100%，结束交易退出策略

    def signal(self, mid_price: float, grid_height: float):
        df = self.df
        entrys, exits, positions = {}, {}, {}
        res_df = pd.DataFrame()
        res_df[['t', 'o', 'h', 'l', 'c', 'v']] = df[['t', 'o', 'h', 'l', 'c', 'v']]
        res_df['date'] = df.index
        last_position = 0
        is_first_buy = True
        last_grid_price = 0
        for i, (index, row) in enumerate(res_df.iterrows()):
            entrys[index], exits[index] = False, False
            positions[index] = last_position
            #第一次买入，直接买50%仓位
            if is_first_buy and row['c'] < mid_price:
                # 判断所属档位
                n = (mid_price - row['c']) // grid_height
                is_first_buy = False
                entrys[index] = True
                positions[index] = 0.5
                last_position = positions[index]
                last_grid_price = mid_price - grid_height * n
            # 非第一次买，当价格下降n格时买入0.1*n个仓位,上升n格卖出0.1*n个仓位
            elif not is_first_buy:
                # n = abs(row['c'] - last_grid_price)//grid_height
                #买
                if row['c'] <= last_grid_price - grid_height and last_position <= 0.9:
                    entrys[index] = True
                    positions[index] = last_position + 0.1
                    last_grid_price = last_grid_price - grid_height
                #卖    
                if row['c'] >= last_grid_price + grid_height and last_position >= 0.1:
                    exits[index] = True
                    positions[index] = last_position - 0.1
                    last_grid_price = last_grid_price + grid_height
                #仓位归零,退出交易 
                if last_position == 0:
                    break
                ###    
                last_position = positions[index]

        res_df['pos'] = positions.values()
        res_df['entry'] = entrys.values()
        res_df['exit'] = exits.values()
        self.df = res_df
        return self

    ###
    # vectorbt快速回测
    def backtest(self, init_cash=100, fees=0):
        pf = vectorbt.Portfolio.from_signals(
            self.df['c'],
            entries=self.df['entry'],
            exits=self.df['exit'],
            price=self.df['c'],
            init_cash=init_cash,
            size=self.df['pos'].apply(lambda x: x * 100),
            fees=fees
        )
        return pf

    # 展示k线
    def show_kline(self):
        res_df = self.df.copy()
        fig = go.Figure(layout=go.Layout(width=1200, height=600))
        fig.add_candlestick(
            name='kline',
            x=res_df.index,
            open=res_df['o'],
            high=res_df['h'],
            low=res_df['l'],
            close=res_df['c']
        )
        fig.data[0].increasing.fillcolor = 'green'
        fig.data[0].increasing.line.color = 'green'
        fig.data[0].decreasing.fillcolor = 'red'
        fig.data[0].decreasing.line.color = 'red'
        # #添加MA_S MA_L
        # fig.add_scatter(x=res_df.index, y=res_df['ma_s'], mode='lines', line=dict(color='blue', width=1), name="MA_S")
        # fig.add_scatter(x=res_df.index, y=res_df['ma_l'], mode='lines', line=dict(color='orange', width=1), name="MA_L")
        fig.update(layout_xaxis_rangeslider_visible=False)
        fig.show()

In [332]:
# 先获取最近1w条数据，存入data.csv
StrategyFrame(pd.DataFrame()).get_last_data(symbol='FDUSD/USDT', count=1)

In [328]:
data = pd.read_csv('./data.csv', parse_dates=['datetime'])
data.set_index('datetime', inplace=True)
SF = StrategyFrame(data)
SF = SF.signal(0.9995, 0.0001)
# print(SF.df)
# SF.show_kline()
pf = SF.backtest(init_cash=100, fees=0)
pf.stats()
# SF.show_kline()

Start                               2024-08-17 06:57:00
End                                 2024-10-25 17:36:00
Period                                 69 days 10:40:00
Start Value                                       100.0
End Value                                       109.984
Total Return [%]                                  9.984
Benchmark Return [%]                           -0.09994
Max Gross Exposure [%]                        98.883478
Total Fees Paid                                     0.0
Max Drawdown [%]                                0.13831
Max Drawdown Duration                   8 days 13:22:00
Total Trades                                       2896
Total Closed Trades                                2896
Total Open Trades                                     0
Open Trade PnL                                      0.0
Win Rate [%]                                  95.959945
Best Trade [%]                                 0.070056
Worst Trade [%]                                -

In [320]:
## 策略调参
## 这次使用optuna， 安装 pip install optuna
import optuna

data = pd.read_csv('./data.csv', parse_dates=['datetime'])
data.set_index('datetime', inplace=True)


def objective(trial):
    n = trial.suggest_int('n', 9990,10000)
    m = trial.suggest_int('m', 1, 10)
    pf = StrategyFrame(data).signal(n/10000, m/10000).backtest(init_cash=100, fees=0)
    return -pf.total_return()


In [329]:
study = optuna.create_study()  # 使用optuna创建Study
study.optimize(objective, n_trials=100)  # 设置Study的优化次数
study.best_params

[I 2024-10-25 20:40:09,355] A new study created in memory with name: no-name-afe2d503-7441-4144-8fd3-2599f86e033f
[I 2024-10-25 20:40:10,795] Trial 0 finished with value: -0.0010399999999997434 and parameters: {'n': 9991, 'm': 9}. Best is trial 0 with value: -0.0010399999999997434.
[I 2024-10-25 20:40:12,527] Trial 1 finished with value: -0.09983999999999839 and parameters: {'n': 9995, 'm': 1}. Best is trial 1 with value: -0.09983999999999839.
[I 2024-10-25 20:40:13,964] Trial 2 finished with value: -0.00046999999999982836 and parameters: {'n': 9996, 'm': 10}. Best is trial 1 with value: -0.09983999999999839.
[I 2024-10-25 20:40:15,395] Trial 3 finished with value: -0.004699999999999918 and parameters: {'n': 9993, 'm': 3}. Best is trial 1 with value: -0.09983999999999839.
[I 2024-10-25 20:40:16,822] Trial 4 finished with value: 0.0007200000000000273 and parameters: {'n': 9998, 'm': 8}. Best is trial 1 with value: -0.09983999999999839.
[I 2024-10-25 20:40:18,259] Trial 5 finished with v

{'n': 9993, 'm': 1}

In [330]:
pf = SF.signal(9993/10000, 1/10000).backtest(init_cash=100, fees=0)
pf.stats()

Start                               2024-08-17 06:57:00
End                                 2024-10-25 17:36:00
Period                                 69 days 10:40:00
Start Value                                       100.0
End Value                                        175.64
Total Return [%]                                  75.64
Benchmark Return [%]                           -0.09994
Max Gross Exposure [%]                         98.89011
Total Fees Paid                                     0.0
Max Drawdown [%]                               0.115097
Max Drawdown Duration                   2 days 03:45:00
Total Trades                                      10080
Total Closed Trades                               10079
Total Open Trades                                     1
Open Trade PnL                                      0.0
Win Rate [%]                                  97.539438
Best Trade [%]                                 0.030021
Worst Trade [%]                                -

In [331]:
pf.orders.records_readable

,Order Id,Column,Timestamp,Size,Price,Fees,Side
0,0,0,2024-08-20 13:46:00,50.0,0.9992,0.0,Buy
1,1,0,2024-08-20 13:48:00,50.0,0.9994,0.0,Sell
2,2,0,2024-08-20 13:55:00,50.0,0.9993,0.0,Buy
3,3,0,2024-08-20 13:56:00,50.0,0.9994,0.0,Sell
4,4,0,2024-08-20 13:57:00,50.0,0.9993,0.0,Buy
...,...,...,...,...,...,...,...
20154,20154,0,2024-10-24 22:19:00,10.0,0.9996,0.0,Buy
20155,20155,0,2024-10-24 23:46:00,10.0,0.9998,0.0,Sell
20156,20156,0,2024-10-25 01:12:00,10.0,0.9996,0.0,Buy
20157,20157,0,2024-10-25 02:25:00,10.0,0.9998,0.0,Sell
